# **E-Commerce Analysis: Customer Churn & Delivery Probelms**
An analysis of Brazillian E-Commerce transactions made at [Olist Store](www.olist.com).

**Created By:** *Fritz Immanuel & Gerard Louis Howan* (JCDS 2502)

## **> Introduction**

<hr>

[Olist](www.olist.com) is a Brazilian-based technology company that provides a platform to help small and medium-sized businesses sell their products on major online marketplaces. Founded in 2015, Olist acts as a bridge between merchants and e-commerce platforms like Amazon, Mercado Libre, and Magalu, enabling sellers to manage their inventory, listings, and orders from a single interface.

By offering logistics, catalog optimization, and customer service tools, Olist simplifies the process of selling online for retailers who might otherwise struggle with the complexity of marketplace requirements. Its goal is to democratize access to e-commerce and boost the visibility of smaller sellers in competitive digital environments.

### **Context**

Olist noticed something isn't right and decided to hire us, a team of Data Scientists, to take a deeper look. By giving us this dataset, we are tasked to look for problems with their business. So we decided to take a look at how their customers are doing and if theres anything wrong that we can improve. Most certainly, it is about customer churning. As we all know, an E-commerce platform is nothing without its customers. For this analysis, we will determine that any **customer who has not made any second order within 6 months** as a **churned customer**.

**Target:**<br>
0 => Staying Customer / non-churn<br>
1 => Leaving Customer / Churn<br>

### **Problem Statement**

With the rise of e-commerce, competition becomes fiercer than ever. With the amount of e-commerce platforms there are today, it provides options for customers to find which is more suited to them. This is however not a good news for the other platforms who are less preferred by customers. In order to have the upper hand, the companies fight to deliver the best customer experience and attract more customers.

There are a number of ways to achieve this, one of them being a full out marketing campaigns. While costing the companies a fortune, these campaigns does not really provide the best ROI. This means with the high costs, it does not translate to more attracted customers. Rather than spending all that costs for uncertain returns, we can try and retain existing customers. Retaining existing customers in a lot of ways takes less effort. Why? We can provide incentives such as discount vouchers to these customers to keep them coming back. Aside from that, we can always listen to customer complaints and start fixing things, in which results in customer loyalty.

### **Goals**

TODO

### **Analytic Approach**

TODO

### **Metric Evaluation**

TODO

## **> Data Understanding**
<hr>

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import folium

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None) # display all columns, without limits

In [2]:
# load dataset
customers_df = pd.read_csv('dataset/olist_customers_dataset.csv')

orders_df = pd.read_csv('dataset/olist_orders_dataset.csv')
order_items_df = pd.read_csv('dataset/olist_order_items_dataset.csv')
order_payments_df = pd.read_csv('dataset/olist_order_payments_dataset.csv')
order_reviews_df = pd.read_csv('dataset/olist_order_reviews_dataset.csv')

products_df = pd.read_csv('dataset/olist_products_dataset.csv')
pcateg_translation_df = pd.read_csv('dataset/product_category_name_translation.csv')

sellers_df = pd.read_csv('dataset/olist_sellers_dataset.csv')

geolocation_df = pd.read_csv('dataset/olist_geolocation_dataset.csv')

In [3]:
# translate product categories
final_products_df = products_df.merge(pcateg_translation_df, on='product_category_name', how='left')
final_products_df = final_products_df.drop(columns=['product_category_name'])
final_products_df = final_products_df.rename(columns={'product_category_name_english': 'product_category_name'})
final_products_df

,product_id,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,96bd76ec8810374ed1b65e291975717f,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,27.0,261.0,1.0,371.0,26.0,4.0,26.0,baby
4,9dc1a7de274444849c219cff195d0b71,37.0,402.0,4.0,625.0,20.0,17.0,13.0,housewares
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,45.0,67.0,2.0,12300.0,40.0,40.0,40.0,furniture_decor
32947,bf4538d88321d0fd4412a93c974510e6,41.0,971.0,1.0,1700.0,16.0,19.0,16.0,construction_tools_lights
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,50.0,799.0,1.0,1400.0,27.0,7.0,27.0,bed_bath_table
32949,83808703fc0706a22e264b9d75f04a2e,60.0,156.0,2.0,700.0,31.0,13.0,20.0,computers_accessories


In [4]:
# order related data
final_order_df = orders_df.merge(order_items_df, on='order_id', how='left')
final_order_df = final_order_df.merge(order_payments_df, on='order_id', how='left')
# final_order_df = final_order_df.merge(order_reviews_df, on='order_id', how='left')
final_order_df = final_order_df.merge(final_products_df, on='product_id', how='left')
final_order_df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,1.0,credit_card,1.0,18.12,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,3.0,voucher,1.0,2.00,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,2.0,voucher,1.0,18.59,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,1.0,boleto,1.0,141.46,29.0,178.0,1.0,400.0,19.0,13.0,19.0,perfumery
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,1.0,credit_card,3.0,179.12,46.0,232.0,1.0,420.0,24.0,19.0,21.0,auto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118429,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,1.0,f1d4ce8c6dd66c47bbaa8c6781c2a923,1f9ab4708f3056ede07124aad39a2554,2018-02-12 13:10:37,174.90,20.10,1.0,credit_card,3.0,195.00,52.0,828.0,4.0,4950.0,40.0,10.0,40.0,baby
118430,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,1.0,b80910977a37536adeddd63663f916ad,d50d79cb34e38265a8649c383dcffd48,2017-09-05 15:04:16,205.99,65.02,1.0,credit_card,5.0,271.01,51.0,500.0,2.0,13300.0,32.0,90.0,22.0,home_appliances_2
118431,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,1.0,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59,1.0,credit_card,4.0,441.16,59.0,1893.0,1.0,6550.0,20.0,20.0,20.0,computers_accessories
118432,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,2.0,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59,1.0,credit_card,4.0,441.16,59.0,1893.0,1.0,6550.0,20.0,20.0,20.0,computers_accessories


In [5]:
final_order_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118434 entries, 0 to 118433
Data columns (total 26 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       118434 non-null  object 
 1   customer_id                    118434 non-null  object 
 2   order_status                   118434 non-null  object 
 3   order_purchase_timestamp       118434 non-null  object 
 4   order_approved_at              118258 non-null  object 
 5   order_delivered_carrier_date   116360 non-null  object 
 6   order_delivered_customer_date  115037 non-null  object 
 7   order_estimated_delivery_date  118434 non-null  object 
 8   order_item_id                  117604 non-null  float64
 9   product_id                     117604 non-null  object 
 10  seller_id                      117604 non-null  object 
 11  shipping_limit_date            117604 non-null  object 
 12  price                         